<a href="https://colab.research.google.com/github/taaha3244/atom-bot/blob/main/RAGQdrant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pypdf openai transformers Chroma langchain langchain_openai chromadb
!pip install --upgrade --quiet  qdrant-client

In [16]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/atomcamp.pdf")
pages = loader.load_and_split()
len(pages)
pages[0]

Document(page_content="We are a young ed-tech platform focused on | Data Science and AI | . We \ncommenced operations in 2021. Since then, we have experienced a continuous \nupward trajectory, both within the ed-tech ecosystem and within our own team. |\nT wo years down the lane, we are proud to have a team of 35 employees, with the\ndistinction of | 75% | of them being women. This remarkable achievement \nreflects the efforts of our founders, Dr. Naveed Iftikhar and Nida Mehmood, who \nhave carefully curated this team with dedication. | Our | mission | , focused on \nproviding accessible and all-inclusive education in the fields of Data Science and \nAI, has witnessed exponential growth with each successive cohort of bootcamps. \n| Through these | training initiatives | , we have nurtured a community of lifelong \nlearners who not only contribute to the Data and AI ecosystem but have also \nhoned the art of effective collaboration between diverse entities. | By offering \nimmersive | 

In [17]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(

    chunk_size=1000,
    chunk_overlap=200
)

texts = text_splitter.split_documents(pages)

print(texts[0])
print(texts[1])

page_content='We are a young ed-tech platform focused on | Data Science and AI | . We \ncommenced operations in 2021. Since then, we have experienced a continuous \nupward trajectory, both within the ed-tech ecosystem and within our own team. |\nT wo years down the lane, we are proud to have a team of 35 employees, with the\ndistinction of | 75% | of them being women. This remarkable achievement \nreflects the efforts of our founders, Dr. Naveed Iftikhar and Nida Mehmood, who \nhave carefully curated this team with dedication. | Our | mission | , focused on \nproviding accessible and all-inclusive education in the fields of Data Science and \nAI, has witnessed exponential growth with each successive cohort of bootcamps. \n| Through these | training initiatives | , we have nurtured a community of lifelong \nlearners who not only contribute to the Data and AI ecosystem but have also \nhoned the art of effective collaboration between diverse entities. | By offering' metadata={'source': '/

In [18]:
from langchain_openai import OpenAIEmbeddings
from google.colab import userdata
embeddings_model = OpenAIEmbeddings(openai_api_key=userdata.get('openai_api_key'))

In [19]:

from langchain_community.vectorstores import Qdrant


url = userdata.get('qdrant_url')
qdrant_api_key = userdata.get('qdrant')
qdrant = Qdrant.from_documents(
    texts,
    embeddings_model,
    url=url,
    prefer_grpc=True,
    api_key=qdrant_api_key,
    collection_name="atom_bot_embeddings",
)


In [52]:
question= "What is your name??"
context = qdrant.similarity_search(question)
context[0]

Document(page_content='September at PNCA (Pakistan National Council of Arts), F5, Islamabad. The \nfestival was a celebration of Art Therapy and self expression. The festival was a \nsuccess as it allowed men, women and children, to showcase their expression \nand thoughts through colors and clay. | Women in CS-Career Development \nWorkshop | This free workshop guided approximately 20 female students and \ngraduates of Computer Science on how to build a competent profile, land a \ndream job, and explore a range of different opportunities available for them at \nnational and international levels. This workshop focused on: | CV/Resume Writing\n| Interview Skills | Potential Career Opportunities (theyâ€™re numerous!) | Closed\n| Meet the Trainer | Trainer | Ã— | Mahnoor Fatima | Global T echnical Product \nManager (Germany) | Education: Bachelors in Computer Sciences from FAST NU, \nIslamabad | T echnical Software Development Training from TU Munich | Close |', metadata={'page': 4, 'sourc

In [55]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", """"You are a helpful AI bot  and belong to an online education platform named Atom camp. Your name is atom-bot.
                       Use the following pieces of context to answer the questions at the end.If you don't know the
                       answer, just say don't know. do not try to make up the answer.   {context}        """""),


        ("human", "Hello, how are you doing?"),
        ("ai", "I'm doing well, thanks!"),
        ("human", "{question}"),
    ]
)



In [46]:
from langchain.prompts import PromptTemplate
prompt=PromptTemplate(
    template=""""You ara a chatbot named 'Atom-bot' and belong to an online education platform named Atom camp.
    Your purpose is to spread positive image of the institute and give information to the user.Use the following pieces of context to answer the questions at the end.If
    you don't know the answer, just say don't know. do not try to make up the answer.

    <context>
    {context}
    </context>

    Question: {question}
    Helpful Answer,formatted in markdown:""",
    input_variables=["context","question"]
)


In [54]:
from langchain.chains import LLMChain
from langchain.llms import OpenAI

llm=OpenAI(openai_api_key=userdata.get('openai_api_key'))
chain=LLMChain(llm=llm,prompt=chat_template)

result=chain({
    "question":question,
    "context":"/n".join([doc.page_content for doc in context])
})

print(result['text'])


AI: My name is atom-bot. I am a helpful AI bot belonging to an online education platform.


Accessing directly from qdrant cloud

In [65]:
from qdrant_client import QdrantClient

qdrant_client = QdrantClient(url=userdata.get('qdrant_url'))


In [64]:
search_result = qdrant_client.search(
    collection_name="atom_bot_embeddings",
    query_vector='Who is Fizza?',  # Your query vector here
    query_filter=None # Use this if you have specific filters # How many results you want
)
print(search_result)


ValidationError: 3 validation errors for SearchRequest
vector.NamedSparseVector
  Input should be a valid dictionary or instance of NamedSparseVector [type=model_type, input_value='Who is Fizza?', input_type=str]
    For further information visit https://errors.pydantic.dev/2.6/v/model_type
vector.NamedVector
  Input should be a valid dictionary or instance of NamedVector [type=model_type, input_value='Who is Fizza?', input_type=str]
    For further information visit https://errors.pydantic.dev/2.6/v/model_type
vector.list[float]
  Input should be a valid list [type=list_type, input_value='Who is Fizza?', input_type=str]
    For further information visit https://errors.pydantic.dev/2.6/v/list_type